In [1]:
import networkx as nx
import numpy as np
import powerlaw  
from tabulate import tabulate  
import os
from pprint import pprint

## P4.7 (2.5 points)

In [2]:
def get_graph(graph_filepath: str):
    G = None
    if graph_filepath.endswith('.mtx'):
        G = nx.read_adjlist(graph_filepath)  
    elif graph_filepath.endswith('.edges'):
        G = nx.read_edgelist(graph_filepath)
    else:
        ValueError("Wrong graph type!")
        
    return G 


In [ ]:
import networkx as nx
import powerlaw
import numpy as np
import matplotlib.pyplot as plt

def calculate_network_metrics(G, name):
    N = G.number_of_nodes()
    E = G.number_of_edges()
    avg_degree = sum(dict(G.degree()).values()) / N
    assortativity = nx.degree_assortativity_coefficient(G)
    
    if nx.is_connected(G):
        avg_shortest_path_length = nx.average_shortest_path_length(G)
    else:
        largest_cc = max(nx.connected_components(G), key=len)
        subgraph = G.subgraph(largest_cc).copy()
        avg_shortest_path_length = nx.average_shortest_path_length(subgraph)

    degrees = [d for _, d in G.degree()]
    
    try:
        fit = powerlaw.Fit(degrees, discrete=True)
        is_power_law = fit.distribution_compare('power_law', 'exponential')[0] > 0
        alpha = fit.power_law.alpha if is_power_law else None
    except Exception:
        alpha = None  
    
    return {
        "Network's Name": name,
        "N": N,
        "E": E,
        "⟨k⟩": round(avg_degree, 2),
        "α": round(alpha, 2) if alpha else "Not a power-law",
        "ℓ": round(avg_shortest_path_length, 2),
        "r": round(assortativity, 2)
    }



In [14]:
directory_path = './graphs/' 
networks = dict()

for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    
    if os.path.isfile(file_path):
        networks[filename.split('.')[0]] = file_path

pprint(networks)

{'bn-cat-mixed-species_brain_1': './graphs/bn-cat-mixed-species_brain_1.edges',
 'ca-GrQc': './graphs/ca-GrQc.mtx',
 'ca-sandi_auths': './graphs/ca-sandi_auths.mtx',
 'inf-USAir97': './graphs/inf-USAir97.mtx',
 'road-chesapeake': './graphs/road-chesapeake.mtx'}


**Assortativity Interpretation (r)**:
- *Positive r*: Nodes tend to connect to nodes with similar degrees (e.g., social networks).
- *Negative r*: High-degree nodes tend to connect with low-degree nodes (e.g., some biological or technological networks).

**ℓ** - Average Path Length

In [15]:
def add_original_metrics(name, N, E, k, alpha, l, r):
    return {
        "Network's Name": name,
        "N": N,
        "E": E,
        "⟨k⟩": k,
        "α": alpha,
        "ℓ": l,
        "r": r
    }

original_metrics = [
    ['ca-GrQc', 5_000, 14_000, 5, '-', '-', 0.66],
    ['road-chesapeake', 39, 170, 8, '-', '-', -0.37],
    ['bn-cat-mixed-species_brain_1', 65, 1_100, 35, '-', '-', 0.01],
    ['ca-sandi_auths', 86, 124, 2, '-', '-', -0.25],
    ['inf-USAir97', 332, 2_100, 12, '-', '-', -0.20]
]

final_original_metrics = []

for metric in original_metrics:
    final_original_metrics.append(add_original_metrics(*metric))

In [ ]:
from tqdm import tqdm

metrics = []
for name, graph_path in tqdm(networks.items()):
    metrics.append(calculate_network_metrics(get_graph(graph_path), name))
    

print(tabulate(metrics, headers="keys", tablefmt="fancy_grid"))

 20%|██        | 1/5 [00:05<00:23,  5.80s/it]

Calculating best minimal value for power law fit
Calculating best minimal value for power law fit
Calculating best minimal value for power law fit
Calculating best minimal value for power law fit


Values less than or equal to 0 in data. Throwing out 0 or negative values
100%|██████████| 5/5 [00:06<00:00,  1.30s/it]

Calculating best minimal value for power law fit
╒══════════════════════════════╤══════╤═══════╤═══════╤═════════════════╤══════╤═══════╕
│ Network's Name               │    N │     E │   ⟨k⟩ │ α               │    ℓ │     r │
╞══════════════════════════════╪══════╪═══════╪═══════╪═════════════════╪══════╪═══════╡
│ ca-GrQc                      │ 4164 │ 13428 │  6.45 │ 2.04            │ 6.05 │  0.64 │
├──────────────────────────────┼──────┼───────┼───────┼─────────────────┼──────┼───────┤
│ road-chesapeake              │   45 │   176 │  7.82 │ 3.15            │ 1.85 │ -0.32 │
├──────────────────────────────┼──────┼───────┼───────┼─────────────────┼──────┼───────┤
│ bn-cat-mixed-species_brain_1 │   65 │   730 │ 22.46 │ Not a power-law │ 1.7  │ -0.03 │
├──────────────────────────────┼──────┼───────┼───────┼─────────────────┼──────┼───────┤
│ ca-sandi_auths               │   97 │   210 │  4.33 │ 2.53            │ 2.68 │ -0.24 │
├──────────────────────────────┼──────┼───────┼───────┼──────

* road (road-chesapeake) - we can see short path length due to efficient road layout design, which is typical for road graphs. Average degree is rather high which is not that typical, but we have to bear in mind, that this graph is rather small and represents road network of small city. That can also explain negative assorativity coefficient, which also is not that typical for road graphs -maybe it is becuase of that we have here a main road, and others attach to it. 

* biological (bn-cat-mixed-species_brain_1) - Here we have super dense graphs, which is typical for biological networks, with short path length neutral assortativity. It is all explained by evolutionary optimization for speed and efficiency.

* social (ca-sandi_auths, ca-GrQc) - here both networks represent social networks, but one is collaboration network while the other represent authorship network. Both of them represent quite high average degree. The main difference is in assorativity - ca-GrQc is strongly assortative, while ca-sandi_auths is disassortative, meaning hubs tend to connect with less-connected nodes. It can be explained with the fact, that  ca-GrQc can have many collaborations between equally active researchers, while ca-sandi_auths can have senior researchers working more with less-experienced ones.  Ca-sandi_auths has much shorter paths, suggesting a more closely-knit structure, what can be explained in the same way.

* infrastructure (inf-USAir97) - represents rather short paths and hubs, similar to other infrastructure networks. However, it shows slight disassortativity, while road networks typically have more disassortative mixing due to their hierarchical structure. Apparently here major airports often connect with other major airports, leading to a less hierarchical structure, rather than typical,  where major hubs connect to smaller nodes. However, that was the smallest network that was possible to choose on the website, and that can expain this phenomena.

original information from website:

In [8]:
print(tabulate(final_original_metrics, headers="keys", tablefmt="fancy_grid"))

╒══════════════════════════════╤══════╤═══════╤═══════╤═════╤═════╤═══════╕
│ Network's Name               │    N │     E │   ⟨k⟩ │ α   │ ℓ   │     r │
╞══════════════════════════════╪══════╪═══════╪═══════╪═════╪═════╪═══════╡
│ ca-GrQc                      │ 5000 │ 14000 │     5 │ -   │ -   │  0.66 │
├──────────────────────────────┼──────┼───────┼───────┼─────┼─────┼───────┤
│ road-chesapeake              │   39 │   170 │     8 │ -   │ -   │ -0.37 │
├──────────────────────────────┼──────┼───────┼───────┼─────┼─────┼───────┤
│ bn-cat-mixed-species_brain_1 │   65 │  1100 │    35 │ -   │ -   │  0.01 │
├──────────────────────────────┼──────┼───────┼───────┼─────┼─────┼───────┤
│ ca-sandi_auths               │   86 │   124 │     2 │ -   │ -   │ -0.25 │
├──────────────────────────────┼──────┼───────┼───────┼─────┼─────┼───────┤
│ inf-USAir97                  │  332 │  2100 │    12 │ -   │ -   │ -0.2  │
╘══════════════════════════════╧══════╧═══════╧═══════╧═════╧═════╧═══════╛


From the comparison with original data we can clearly see, that the data gathered directly from graphs is very similar to the one collected from the website. In case od nodes and edges data from website has its values rounded, that why it is not detailed like to one taken from graphs. Assortativity looks almost the same, but there are some slight differences in average degree.